In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777) 
sample = " If you want you"
idx2char = list(set (sample)) #index -> char
char2idx = {c:i for i , c in enumerate(idx2char)}

In [3]:
dic_size = len(char2idx) #RNN input size
num_classes = len(char2idx)
hidden_size = len(char2idx) #output size
batch_size = 1
sequence_length = len(sample) -1
learning_rate = 0.1
rnn_hidden_size = len(char2idx)

In [4]:
sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]] # 0~n-1
y_data = [sample_idx[1:]] # 1 ~ n

In [5]:
X = tf.placeholder(tf.int32,[None, sequence_length]) #X one-hot
Y = tf.placeholder(tf.int32,[None, sequence_length]) #Y Label

In [6]:
X_one_hot = tf.one_hot(X, num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

X_for_fc  = tf.reshape(outputs, [-1, rnn_hidden_size])

In [7]:
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [8]:
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
softmax_outputs = tf.matmul(outputs, softmax_w) + softmax_b

In [9]:
#reshape out for squence loss
outputs = tf.reshape(softmax_outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights= weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(500):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict = { X: x_data})
        print( i, "loss: ", l, "prediction: ", result, "true Y: ", y_data)
        
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss:  2.38586 prediction:  [[3 2 3 3 3 3 4 3 3 3 5 5 3 3 5]] true Y:  [[6, 1, 4, 3, 2, 8, 4, 7, 9, 0, 5, 4, 3, 2, 8]]
	Prediction str:  yoyyyy yyyttyyt
1 loss:  2.15577 prediction:  [[2 4 4 4 4 4 4 4 4 4 4 4 4 4 4]] true Y:  [[6, 1, 4, 3, 2, 8, 4, 7, 9, 0, 5, 4, 3, 2, 8]]
	Prediction str:  o              
2 loss:  2.26664 prediction:  [[3 2 2 3 3 3 4 3 3 3 3 3 3 3 3]] true Y:  [[6, 1, 4, 3, 2, 8, 4, 7, 9, 0, 5, 4, 3, 2, 8]]
	Prediction str:  yooyyy yyyyyyyy
3 loss:  1.9542 prediction:  [[3 1 3 3 3 3 2 3 3 3 3 3 3 3 3]] true Y:  [[6, 1, 4, 3, 2, 8, 4, 7, 9, 0, 5, 4, 3, 2, 8]]
	Prediction str:  yfyyyyoyyyyyyyy
4 loss:  1.93872 prediction:  [[3 1 2 3 3 8 2 3 3 3 3 3 3 3 8]] true Y:  [[6, 1, 4, 3, 2, 8, 4, 7, 9, 0, 5, 4, 3, 2, 8]]
	Prediction str:  yfoyyuoyyyyyyyu
5 loss:  1.79107 prediction:  [[1 1 4 3 2 8 4 3 2 8 3 4 3 8 8]] true Y:  [[6, 1, 4, 3, 2, 8, 4, 7, 9, 0, 5, 4, 3, 2, 8]]
	Prediction str:  ff you youy yuu
6 loss:  1.57867 prediction:  [[1 1 4 3 2 8 4 3 4 8 5 4 3 8 8]] true Y: